# Die Pensionskasse

Die Pensionskasse ist nach dem Kapitaldeckungsverfahren finanziert. Das heisst,
jeder Versicherte der Pensionskassen spart ein individuelles Guthaben an,
aus dem die Rente finanziert wird.

Aktuell (Februar 2024) wird das angesparte Guthaben mit einem gesetzlich
festgeschriebenen Umwandlungssatz von 6.8% 
([Art. 14 Abs. 2 BVG](https://www.fedlex.admin.ch/eli/cc/1983/797_797_797/de#art_14))
in eine Rente umgerechnet. Das bedeutet, dass jedes Jahr eine Rente in
der Höhe von 6.8% des bei Erreichen des Rentenalters angesparten
Kaptials ausbezahlt wird.

## Wie lange reicht das angesparte Kapital theoretisch?

Das scheint eine relativ einfach zu beantwortende Frage zu sein.
Es sind 6.8% des angesparten Kapitals zu berechnen. Anschliessend ist
dieser Betrag so oft vom angesparten Kapital in Abzug zu bringen, bis
nichts mehr übrig ist.

In [4]:
counter = 0
savings = 100
pension = 6.8

while savings > 0:
    savings -= pension
    counter += 1

print(f'Die Ersparnisse reichen für {counter} Jahre.')

Die Ersparnisse reichen für 15 Jahre.


So weit so gut. Aber wie lange sollten die Ersparnisse tatsächlich
reichen? Dazu muss auf die statistische Restlebenserwartung bei
Erreichen des Rentenalters abgestellt werden. Diese findet sich in den
sogenannten *Sterbetafeln*. Diese werden, nach Geschlechtern getrennt,
durch das Bundesamt für Statistik zur Verfügung gestellt 
([Männer](https://www.bfs.admin.ch/bfs/de/home/statistiken/bevoelkerung/geburten-todesfaelle/lebenserwartung.assetdetail.27225580.html),
[Frauen](https://www.bfs.admin.ch/bfs/de/home/statistiken/bevoelkerung/geburten-todesfaelle/lebenserwartung.assetdetail.27225585.html)).

Gemäss diesen Tabellen haben Männer, welche 2022 65 Jahre alt geworden
sind, eine Restlebenserwartung von 19.8 Jahren, Frauen gar eine solche
von 22.5 Jahren. Den Männern fehlen nach obiger Berechnung somit Kapital
für 4.8 Jahre Rente, den Frauen für 7.5 Jahre.

## Wie lange reicht das angesparte Kapital praktisch?

Das angesparte Kapital wirft auch nach
erreichen des Rentenalters eine Rendite ab. Damit muss die Frage geklärt
werden, wie hoch die Verzinsung sein muss, damit unter Einhaltung des
gesetzlichen Umwandlungssatzes bis zum Lebensende eine Rente ausbezahlt
werden kann.

Um diesen Zinssatz zu ermitteln, wird rechnerisch per Ende Jahr das noch
vorhandene Kapital verzinst und dann von diesem Betrag die jährliche
Rente in Abzug gebracht.

Der Zinssatz wird, beginnend bei 0.1%, so lange um 0.1 Prozentpunkte
erhöht, bis das Geld für eine Rente bis zum Lebensende reicht.

In [4]:
savings = 100
pension = 6.8
counter = int(19.8 + 22.5) // 2 + 1
remaining_savings = [savings - pension * i
                     for i in range(counter)]
interest = 0.001

"""
Die Mindestverzinsung wird nicht berechnet, sondern iterativ bestimmt.
So muss keine Gleichung gefunden und gelöst werden.
"""
while remaining_savings[-1] <= 0:
    for i in range(1, counter):
        remaining_savings[i] = remaining_savings[i - 1] * (1 + interest) - pension
    interest += 0.001


print(f'Die erforderliche Verzinsung beträgt {100 * interest:.1f} %.')
for savings in remaining_savings:
    print(f'{savings:.2f}', end=' ')

Die erforderliche Verzinsung beträgt 3.6 %.
100.00 96.70 93.28 89.75 86.09 82.30 78.38 74.33 70.13 65.78 61.29 56.63 51.81 46.83 41.67 36.32 30.80 25.07 19.15 13.02 6.68 0.11 

Damit ist die Frage beantwortet, wie hoch die Verzinsung des angesparten
Kaptials sein muss, um im Durchschnitt eine lebenslange Rente
ausschütten zu können.

Nicht geklärt ist allerdings die Frage, ob diese Rendite in der
Vergangenheit tatsächlich erreicht wurde.


## Modellrechnung mit den effektiv erzielten Renditen

Um die Modellrechnung für die Modellrechnung die vorhandenen Daten
importieren zu können, wird in der nächsten Zelle die `pandas` Library
importiert.

In [5]:
import pandas as pd

C:\Users\jcms\AppData\Local\Temp\ipykernel_27380\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Gemäss einer 
[Zusammenstellung der UBS (aufgerufen im Februar
2024)](https://www.ubs.com/microsites/focus/de/markets/2022/pension-fund-performance.html)
wiesen die Pensionskassen in den Jahren von 2006 bis 2023 im Mittel die
folgenden Renditen auf:

In [6]:
actual_performance = pd.read_csv('../data/pk_performance.csv', header=0,
                                 dtype={'Jahr': int, 'Rendite': float})
actual_performance

,Jahr,Rendite
0,2006,5.94
1,2007,1.92
2,2008,-12.84
3,2009,10.59
4,2010,2.83
5,2011,-0.34
6,2012,7.16
7,2013,6.05
8,2014,7.36
9,2015,0.86


In der Folge wird versucht zu zeigen, was das für die Renten bedeutet.
Die Daten der UBS decken den Zeitraum von 2006 bis 2023 ab. Das sind 18
Jahre. Damit Daten für die mittlere Restlebenserwartung zur Verfügung
stehen, werden die Jahre 2002 bis 2005 mit der durchschnittlichen
Rendite gemäss der UBS Datenreihe ergänzt.

In [7]:
durchschnittliche_rendite = actual_performance['Rendite'].mean()
print(
    f'Die effektiv erzielte durchschnittliche Rendite '
    f'gemäss Daten der UBS beträgt {durchschnittliche_rendite:.2f}%.'
    )

Die effektiv erzielte durchschnittliche Rendite gemäss Daten der UBS beträgt 3.16%.


In [8]:
pre_years = [[2002, 3.16],
             [2003, 3.16],
             [2004, 3.16],
             [2005, 3.16],]

df = pd.DataFrame(pre_years, columns=['Jahr', 'Rendite'])

In [9]:
extendend_performance = pd.concat([df, actual_performance], ignore_index=True)
print(extendend_performance.head(6))

   Jahr  Rendite
0  2002     3.16
1  2003     3.16
2  2004     3.16
3  2005     3.16
4  2006     5.94
5  2007     1.92


In [10]:
performance = extendend_performance['Rendite'].to_list()

available_capital = [100]

for i in range(1, len(performance)):
        available_capital.append(available_capital[i - 1] * 
                                 (1 + performance[i]/100) - pension)
        
"""
Für eine bessere Lesbarkeit wird das Ergebnis in ein DataFrame umgewandelt.
"""
    
years = extendend_performance['Jahr'].to_list()

result = pd.DataFrame({'Jahr': years, 'Verfügbares Kapital': available_capital})

print(result.tail(6))

    Jahr  Verfügbares Kapital
16  2018            18.630471
17  2019            13.933852
18  2020             7.703746
19  2021             1.550861
20  2022            -5.397712
21  2023           -12.465978


Das Resultat der Simulationsrechnung zeigt, dass statt der erfordlichen
21 Jahre lediglich deren 20 abgedeckt sind.